<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/code_clones_code2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd /content/gdrive/My\ Drive/code-clones

/content/gdrive/My Drive/code-clones


In [0]:
import xmlextractor
import os
from tqdm import tqdm

In [0]:
DATA_DIR = "data"
XML = "duplicates_marked.xml"
PROJECT = "RxJava-2.x"

projPath = os.path.join(DATA_DIR, PROJECT)
xmlPath = os.path.join(projPath, XML)

duplicates = xmlextractor.parseXml(xmlPath, projPath)

print(len(duplicates))
print(len(list(filter(lambda d: d.exp == 0, duplicates))))
print(len(list(filter(lambda d: d.exp == 1, duplicates))))

5077
3352
1725


In [0]:
%cd /content

/content


In [0]:
!git clone https://github.com/tech-srl/code2vec

Cloning into 'code2vec'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 202 (delta 18), reused 33 (delta 13), pack-reused 148
Receiving objects: 100% (202/202), 6.31 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [0]:
!wget https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz

--2019-03-04 13:19:42--  https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.164.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.164.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1440921240 (1.3G) [application/x-tar]
Saving to: ‘java14m_model.tar.gz’

java14m_model.tar.g 100%[===================>]   1.34G  86.9MB/s    in 17s     

2019-03-04 13:19:59 (80.9 MB/s) - ‘java14m_model.tar.gz’ saved [1440921240/1440921240]



In [0]:
!tar -xvzf java14m_model.tar.gz

models/java14_model/saved_model_iter8.release.data-00000-of-00001
models/java14_model/saved_model_iter8.release.index
models/java14_model/saved_model_iter8.release.meta
models/java14_model/dictionaries.bin


In [0]:
!mv models/ code2vec/

In [0]:
%cd code2vec/

/content/code2vec


In [0]:
#duplicates = duplicates[1189:] #1190 alredy saved

In [0]:
len(duplicates)

5077

In [0]:
import pexpect
import sys
import time
import numpy as np

def expectCodeVectors(child, acc):
  index1 = child.expect(["Code vector:", "Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit"])
  if index1 == 0:
    index2 = child.expect(["Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit", "Original name:"])
    acc.append(child.before)
    if index2 == 0:
      return acc
    elif index2 == 1:
      return expectCodeVectors(child, acc)
  elif index1 == 1:
    return []

def list2vec(l):
  result_acc = []
  for code_vector in acc:
    result_code_vector = []
    update_code_vecor = str(code_vector).replace("\\r\\n", "").replace("'", "")[1:].split()
    for s in update_code_vecor:
      result_code_vector.append(float(s))
    result_acc.append(result_code_vector)
  return result_acc

child = pexpect.spawn ('python3 code2vec.py --load models/java14_model/saved_model_iter8.release --export_code_vectors --predict', timeout=None)

code_vectors = []
child.expect("Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit")

for duplicate in tqdm(duplicates, ascii=True):
  duplicate_code_vectors = []
  for fragment in duplicate.fragments:
    with open("Input.java", 'w') as f:
      f.write(fragment)
    child.sendline("r")
    acc = expectCodeVectors(child, [])
    if acc == []:
      continue
    acc = list2vec(acc)
    duplicate_code_vectors.append(np.mean(acc, axis=0).tolist())
  code_vectors.append({'exp': duplicate.exp, 'vectors': duplicate_code_vectors})

child.sendline("q")
child.expect(pexpect.EOF)
child.close()

100%|##########| 5077/5077 [3:21:41<00:00,  2.17s/it]


In [0]:
len(code_vectors)

5077

In [0]:
import json

JSON_FILE = '/content/gdrive/My Drive/code-clones/csv/rxjava_code2vec_fragments_vectors.json'

with open(JSON_FILE, 'w') as outfile:
    json.dump(code_vectors, outfile)